<a href="https://colab.research.google.com/github/luqmanr/web-scraping/blob/master/instagram-crawler/instagram_crawler_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!python3 -m pip install mtcnn
!python3 -m pip install opencv-contrib-python>=4.1.0
!pip install stdiomask
!apt install chromium-chromedriver
!pip install selenium
!pip install cp
!sudo apt-get install python-yaml
!sudo yum install python-yaml
!apt-get update
!apt install python3-dev python3-pip
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 2.3MB 2.8MB/s 
  Created wheel for stdiomask: filename=stdiomask-0.0.5-cp36-none-any.whl size=3318 sha256=6daa5f2d1f1c760b7aa027c8f76fc6ec82139bfaf98262b73ca5ba777519cc1a
  Stored in directory: /root/.cache/pip/wheels/da/05/d3/b1ac160dd86577b6606330bc1d9fccf7d5dbe1713352a847df
Successfully built stdiomask
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 25 not upgraded.
Need to get 74.4 MB of archives.
After this operation, 264 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/u

In [0]:
import os, time, errno, pickle, stdiomask
import re
import getpass
import json
import csv
import yaml
import sys
import cp
import selenium
from selenium import webdriver
from urllib.request import urlopen
from optparse import OptionParser
from selenium import webdriver  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

maxCrawl = 5
crawlCounter = 0
counter = 0
scroll = 0
folllist = []

WINDOW_SIZE = "1360,768"
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
os.chdir("/content/gdrive/My Drive/")

driver = webdriver.Chrome(
    executable_path="/usr/lib/chromium-browser/chromedriver",
    chrome_options=chrome_options
)

def login_instagram():
    try:
        driver.get("https://www.instagram.com/accounts/login/")
        time.sleep(2)
        driver.find_element_by_name("username").send_keys(login_id)
        driver.find_element_by_name("password").send_keys(password)
        time.sleep(1)
        driver.find_element_by_class_name("L3NKy").click()
        return login_id
    except:
        print("the Xpath of this element (login_instagram) has changed")
        time.sleep(6)
        driver.close

def followerButton():
    try:
        follButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[2]/a/span'
        follower_button = driver.find_element_by_xpath(follButtonXpath)
        printfoll = follower_button.get_attribute('title')
    except ValueError:
        follButtonXpath = '/html/body/div[1]/section/main/div/ul/li[2]/a'
        follower_button = driver.find_element_by_xpath(follButtonXpath)
        pass
    except:
        pass
        print("the Xpath of this element (followerButton) has changed")
    print("button found")
    follValue = printfoll.split(',')
    print(follValue)
    try:
      follvalue0 = follValue[0]
      follvalue1 = follValue[1]
      follvalue = str(follvalue0 + follvalue1)
      print(follvalue)
      follower_button.click()
      return follvalue
    except:
      follvalue0 = follValue[0]
      follvalue = str(follvalue0)
      follvalue = follvalue0
      print(follvalue)
      follower_button.click()
      return follvalue

def follGet(n):
  follmax = 100
  while n <= follmax:
    str_n = str(n)
    try:
      follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_n + ']/div/div[1]/div[2]/div[1]/a'
      follower_name = driver.find_element_by_xpath(follXpath)
      time.sleep(0.2)
      follvalue = (follower_name.text)
      return follvalue
    # except ValueError:
    #   follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_n + ']/div/div[2]/div[1]/div/div/a'
    #   follower_name = driver.find_element_by_xpath(follXpath)
    #   time.sleep(0.2)
    #   follvalue = (follower_name.text)
    #   return follvalue
    except:
      pass
      print("the Xpath of this element (follGet) has changed")
    if n > follmax:
      break
    else:
      pass

def URL():
    urlmax = 100  #maximal userID to take (to avoid account block by Instagram)
    while counter <= urlmax:
        driver.switch_to.window(driver.window_handles[1])
        driver.get("https://www.instagram.com/"+USERstr+"/?__a=1")
        time.sleep(0.3)
        userID=driver.find_element_by_xpath('/html/body/pre')
        user=userID.get_attribute('innerHTML')
        s=user
        y=yaml.load(s)
        l=(y.get("logging_page_id"))
        IGid=(l.split('_'))
        IGidVal = IGid[1]
        driver.switch_to.window(driver.window_handles[0])
        if counter > max:
            print("ID Limit exceded")
            break
        return IGidVal

def myURL():
    driver.switch_to.window(driver.window_handles[1])
    driver.get("https://www.instagram.com/"+login_id+"/?__a=1")
    time.sleep(0.3)
    userID=driver.find_element_by_xpath('/html/body/pre')
    user=userID.get_attribute('innerHTML')
    s=user
    y=yaml.load(s)
    l=(y.get("logging_page_id"))
    IGid=(l.split('_'))
    myIGidVal = IGid[1]
    driver.switch_to.window(driver.window_handles[0])
    return myIGidVal

login_id = input('Your IG Account Username: ')
password = getpass.getpass('Your IG Account Password: ')
login_instagram()
time.sleep(5)

while crawlCounter <= maxCrawl:
  crawlCounter += 1
  user_target = input('Your IG Account Username Target: ')
  driver.get('https://www.instagram.com/'+ user_target + '/')
  time.sleep(3)
  follvalue=followerButton()
  xint = int(follvalue)
  time.sleep(3)

  #scroll
  fBody = driver.find_element_by_xpath("//div[@class='isgrP']")
  while scroll < xint:
      driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', fBody)
      time.sleep(0.1)
      scroll += 1
      if scroll == xint/2:
          print("break")
          break
      else:
          pass

  # driver.execute_script("window.open('');")
  # myIGidVal = str(myURL())
  # print(myIGidVal)
  time.sleep(1)
  for i in range(1, xint):
      #get username
      USERstr = str(follGet(i))
      print(str(i)+USERstr)
      
      #get userid
      # IDstr = str(URL())
      # counter += 1
      # print(IDstr)
      time.sleep(0.5)
      folllist.append(login_id + "," + USERstr)
      # folllist.append(login_id + "," + myIGidVal + "," + USERstr + "," + IDstr)

      if USERstr == "None":
          USERstr = "ID limit exceded"
          break
      else:
          pass
      # if IDstr == "None":
      #     IDstr = "ID limit exceded"
      # else:
      #     pass
  print(folllist)
  time.sleep(3)
  with open('followers(' + user_target + ').csv', 'w', newline='') as csvfile:
      wr = csv.writer(csvfile)
      for follower in folllist:
          wr.writerow([follower])
  driver.execute_script("window.open('');")
  time.sleep(5)
time.sleep(5)
driver.close
driver.close

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: use options instead of chrome_options


Your IG Account Username: tyogotest
Your IG Account Password: ··········
Your IG Account Username Target: tyogo_utomo
button found
['490']
490
break
1pinjamsignal
2markitongs
3teodorusjordanp
4kuaisin
5tyogotest
6paulsoareshutama_94
7sofashabby01
8nathchriist
9surabi052
101eejhonson
11iscaamanda
12idiotic_kidots
13luqmanr
14hollanastax
15inilahniko
16agile_lordax
17coconut.brah
18debukocheng
19dr._doni
20m.rizkyrani
21yourkomrade_
22johnsonmark1821
23dita.irjayana
24syifalaila03
25andreasmaradhana
26rezaavantgarde
27temanriset
28yosep_adi_sutjipto
29nogirlnohehe
30davidhendriawan_
31gigagaharu
32janice.cynthia
33gaby.t_s
34jasa_backdropevent_bali
35bagastheo_mamayo
36frey_santos
37terpalplastikbandung
38marteriacandra
39angelanuriii
40salmaandriani17
41rauf_rafael
42moulitaa
43nuno.luminari
44mentariqorina
45yazid.number7
46sufjrjjfbr23_
47shanandranns
48m.p.f.l_230702
49maritasarah
50ratucumi
51reviewopini123
52mariaflorentina165
53kirana.kasih_
54papolele
55hansanandaa
56fauzianastit